# 🎬 ГОДЖО 30 СЕК — ПРОСТАЯ ВЕРСИЯ БЕЗ COMFYUI

**Что делает этот notebook:**
- Генерирует анимацию с AnimateDiff или статичные кадры
- Применяет upscale через Real-ESRGAN (опционально)
- Интерполирует кадры через RIFE для плавности
- Создает финальное видео

**Инструкция:**
1. Загрузите датасет с моделями в Kaggle
2. Настройте параметры в ячейке ниже
3. Запустите все ячейки по порядку (Run All)

In [ ]:
# === РАННЯЯ НАСТРОЙКА ОКРУЖЕНИЯ / ДИАГНОСТИКА ===
# Устанавливаем важные переменные окружения ДО любых heavy-импортов,
# чтобы избежать сообщений о дублирующей регистрации CUDA-плагинов (cuFFT/cuDNN/cuBLAS) от XLA/JAX/TensorFlow.
import os
import importlib
_changed_env = False

def _set_env_if_needed(k, v):
    global _changed_env
    if os.environ.get(k) != v:
        os.environ[k] = v
        _changed_env = True

# Подавляем TensorFlow info/warnings, отключаем предалокацию XLA клиента
_set_env_if_needed('TF_CPP_MIN_LOG_LEVEL', '3')
_set_env_if_needed('XLA_PYTHON_CLIENT_PREALLOCATE', 'false')
_set_env_if_needed('XLA_PYTHON_CLIENT_MEM_FRACTION', '0.0')
# Принудительно переключаем JAX на CPU, если он установлен — чтобы он не регистрировал GPU-плагины
_set_env_if_needed('JAX_PLATFORM_NAME', 'cpu')
_set_env_if_needed('JAX_PLATFORMS', 'cpu')

# Попытаемся мягко подавить absl-логи (если absl присутствует)
try:
    import absl.logging
    absl.logging._warn_preinit_stderr = False
    absl.logging.set_verbosity(absl.logging.WARNING)
except Exception:
    pass

# Диагностика: есть ли установленные библиотеки, которые могут вызвать XLA/JAX/TensorFlow логирование
_found = {}
for mod in ('jax', 'jaxlib', 'tensorflow', 'torch_xla'):
    _found[mod] = importlib.util.find_spec(mod) is not None

print('ENV diagnostic:')
print('  TF_CPP_MIN_LOG_LEVEL=', os.environ.get('TF_CPP_MIN_LOG_LEVEL'))
print('  XLA_PYTHON_CLIENT_PREALLOCATE=', os.environ.get('XLA_PYTHON_CLIENT_PREALLOCATE'))
print('  XLA_PYTHON_CLIENT_MEM_FRACTION=', os.environ.get('XLA_PYTHON_CLIENT_MEM_FRACTION'))
print('  JAX_PLATFORM_NAME=', os.environ.get('JAX_PLATFORM_NAME'))
print('  JAX_PLATFORMS=', os.environ.get('JAX_PLATFORMS'))
print('Detected potentially conflicting packages:')
for k, v in _found.items():
    print(f'  {k}:', 'present' if v else 'not found')

if _changed_env:
    print('\n⚠️ Важно: переменные окружения изменены — перезапустите kernel (Restart Kernel) перед дальнейшим импортом heavy-библиотек, чтобы изменения вступили в силу и сообщения XLA/TensorFlow не появлялись.')
else:
    print('\nℹ️ Переменные окружения уже установлены.')


In [ ]:
# === KAGGLE SETUP & CHECKS ===
# Эта ячейка автоматически выполняет базовую проверку для запуска в Kaggle и даёт инструкции.
import os
import sys
import textwrap

is_kaggle = any(p.startswith('/kaggle') for p in (os.getcwd(),)) or os.environ.get('KAGGLE_KERNEL_RUN_TYPE') is not None
print_separator()
print('🧭 KAGGLE CHECK')
print_separator(nl_before=False)
print('Running in Kaggle environment:', is_kaggle)

# GPU check (если torch доступен)
try:
    import torch
    cuda_avail = torch.cuda.is_available()
    device_name = torch.cuda.get_device_name(0) if cuda_avail else 'no gpu'
    print(f'GPU available: {cuda_avail} — {device_name}')
except Exception:
    print('Torch not imported yet — GPU check skipped (will be checked again in import cell).')

# Check for Hugging Face token in environment or common Kaggle input path
hf_token = os.environ.get('HUGGINGFACE_HUB_TOKEN') or os.environ.get('HF_TOKEN')
hf_token_file = '/kaggle/input/hf-token/token.txt'
if not hf_token and os.path.exists(hf_token_file):
    try:
        with open(hf_token_file, 'r', encoding='utf-8') as f:
            hf_token = f.read().strip()
            os.environ['HUGGINGFACE_HUB_TOKEN'] = hf_token
            print('✓ Hugging Face token found in /kaggle/input/hf-token/token.txt and set to HUGGINGFACE_HUB_TOKEN')
    except Exception as e:
        print('⚠️ Не удалось прочитать token file:', e)

if hf_token:
    print('✓ Hugging Face token available via environment.')
else:
    print('\n⚠️ Hugging Face token not found.')
    print(textwrap.dedent('''
    Чтобы загрузить модели с Hugging Face в Kaggle, добавьте ваш токен следующим образом:

    1) В интерфейсе Kaggle: Add Data -> создайте dataset с файлом token.txt (содержит ваш токен) и подключите его к ноутбуку как /kaggle/input/hf-token
    2) Либо установите переменную окружения прямо в ноутбуке (выполните в отдельной ячейке):

       import os
       os.environ['HUGGINGFACE_HUB_TOKEN'] = 'ВАШ_ТОКЕН'

    После установки токена перезапустите kernel.
    '''))

# ipywidgets hint
try:
    import ipywidgets  # type: ignore
    print('✓ ipywidgets доступен')
except Exception:
    print('\n⚠️ ipywidgets не установлен — GUI для промптов не будет работать.')
    print('  Установите ipywidgets в отдельной ячейке и перезапустите kernel:')
    print('\n```bash\n!pip install ipywidgets -q\n```\n')

# Quick safe defaults suggestion for Kaggle (optional)
if is_kaggle:
    print('\n💡 Рекомендации для запуска в Kaggle:')
    print('  - Включите GPU в Settings -> Accelerator -> GPU')
    print('  - Для теста используйте небольшие параметры: WIDTH=256, HEIGHT=256, NUM_FRAMES=4, STEPS=15')
    print('  - Если модель не помещается в память, уменьшите WIDTH/NUM_FRAMES или используйте model_cpu_offload() (pipeline уже настраивается с этим флагом)')

print_separator()


In [ ]:
# ============================================
# 📝 НАСТРОЙКИ - МЕНЯЙ ТОЛЬКО ЭТО!
# ============================================
# Константы по умолчанию для промптов (используются в нескольких местах)
DEFAULT_BASE_PROMPT = "cinematic portrait of gojo satoru, white spiky hair, black blindfold, confident expression, anime style, highly detailed, 8k, professional lighting"
DEFAULT_NEGATIVE_PROMPT = "blurry, deformed, low quality, watermark, text, bad anatomy, multiple heads, duplicate"

PROMPT = DEFAULT_BASE_PROMPT
NEGATIVE_PROMPT = DEFAULT_NEGATIVE_PROMPT

# Режим работы
USE_ANIMATEDIFF = True  # True = настоящая анимация, False = статичные кадры + RIFE

# Параметры генерации
WIDTH = 512
HEIGHT = 768
NUM_FRAMES = 16 if USE_ANIMATEDIFF else 8  # AnimateDiff: 16-24, обычный: 8-12
STEPS = 25 if USE_ANIMATEDIFF else 20  # Для анимации нужно больше steps
CFG_SCALE = 7.5 if USE_ANIMATEDIFF else 7
FPS = 8  # FPS для финального видео

# Интерполяция RIFE (опционально)
USE_RIFE = True  # Применить RIFE для ещё более плавной анимации
RIFE_EXP = 4 if USE_ANIMATEDIFF else 5  # AnimateDiff: 4 (16→256), обычный: 5 (8→256)
# Имя файла модели для апскейла (Real-ESRGAN)
UPSCALE_MODEL_NAME = '4x-UltraSharp.pth'
# ============================================

# === PROMPTS — ВАШИ PROMPT'Ы ДЛЯ ИЗОБРАЖЕНИЯ И АНИМАЦИИ (перемещена) ===
# Эта ячейка теперь находится прямо после блока настроек. Редактируйте здесь BASE/MOTION/EXTRA/NEGATIVE
import os
from IPython.display import display, clear_output

# Путь для сохранения промптов
PROMPTS_FILE = os.path.join(os.getcwd(), 'prompts.json')

# Набор пресетов: имя -> (base, motion, extra, negative)
PRESETS = {
    'default': (
        DEFAULT_BASE_PROMPT,
        "turning head, hair flowing, smooth motion",
        "dramatic rim lighting, soft bloom, depth of field",
        DEFAULT_NEGATIVE_PROMPT
    ),
    'slow pan': (
        "cinematic portrait, highly detailed, 8k, beautiful face",
        "slow camera pan left, subtle head turn",
        "soft warm lighting, cinematic",
        "blurry, low quality, watermark, text"
    ),
    'head turn': (
        "close-up portrait, detailed, professional lighting",
        "turning head to left then right, hair movement",
        "rim light, subtle bloom",
        "multiple heads, deformed, watermark"
    ),
    'blinking': (
        "portrait, soft lighting, anime style",
        "subtle blink, small head tilt",
        "soft bokeh, cinematic lighting",
        "blurry, artifact, watermark"
    )
}

# Загружаем пресет (если PROMPT уже задан, пытаемся сопоставить)
def load_preset(name):
    if name in PRESETS:
        base, motion, extra, negative = PRESETS[name]
        return {'BASE_PROMPT': base, 'MOTION_PROMPT': motion, 'EXTRA_PROMPT': extra, 'NEGATIVE_PROMPT': negative}
    return None

# Сохраняем в prompts.json
def save_prompts_file(data, path=PROMPTS_FILE):
    try:
        import json
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"✓ Prompts saved to {path}")
    except Exception as e:
        print('⚠️ Не удалось сохранить prompts.json:', e)

# Загружаем из prompts.json
def load_prompts_file(path=PROMPTS_FILE):
    try:
        import json
        if os.path.exists(path):
            with open(path, 'r', encoding='utf-8') as f:
                return json.load(f)
    except Exception as e:
        print('⚠️ Не удалось загрузить prompts.json:', e)
    return None

# Попытка подключить ipywidgets; если недоступен — выводим fallback инструкции
def create_prompts_gui():
    try:
        import ipywidgets as widgets
    except Exception:
        print('⚠️ ipywidgets не установлен — GUI недоступен. Установите: pip install ipywidgets')
        # Fallback: просто создаём PROMPT из текущ констант
        parts = [p for p in (globals().get('PROMPT', ''), '') if p]
        globals()['PROMPT'] = globals().get('PROMPT', '')
        globals()['NEGATIVE_PROMPT'] = globals().get('NEGATIVE_PROMPT', '')
        print('\nPROMPT (fallback):', globals()['PROMPT'])
        return

    # Виджеты
    preset_dropdown = widgets.Dropdown(options=list(PRESETS.keys()), value='default', description='Preset:')
    base_ta = widgets.Textarea(value=PRESETS['default'][0], description='Base:', layout=widgets.Layout(width='100%', height='80px'))
    motion_ta = widgets.Textarea(value=PRESETS['default'][1], description='Motion:', layout=widgets.Layout(width='100%', height='60px'))
    extra_ta = widgets.Textarea(value=PRESETS['default'][2], description='Extra:', layout=widgets.Layout(width='100%', height='60px'))
    negative_ta = widgets.Textarea(value=PRESETS['default'][3], description='Negative:', layout=widgets.Layout(width='100%', height='60px'))

    save_btn = widgets.Button(description='Save to prompts.json', button_style='success')
    load_btn = widgets.Button(description='Load from prompts.json')
    update_btn = widgets.Button(description='Update PROMPT', button_style='primary')
    out = widgets.Output()

    # Обработчики
    def on_preset_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            vals = load_preset(change['new'])
            if vals:
                base_ta.value = vals['BASE_PROMPT']
                motion_ta.value = vals['MOTION_PROMPT']
                extra_ta.value = vals['EXTRA_PROMPT']
                negative_ta.value = vals['NEGATIVE_PROMPT']

    def on_save_clicked(b):
        data = {
            'BASE_PROMPT': base_ta.value,
            'MOTION_PROMPT': motion_ta.value,
            'EXTRA_PROMPT': extra_ta.value,
            'NEGATIVE_PROMPT': negative_ta.value
        }
        save_prompts_file(data)

    def on_load_clicked(b):
        data = load_prompts_file()
        if data:
            base_ta.value = data.get('BASE_PROMPT', base_ta.value)
            motion_ta.value = data.get('MOTION_PROMPT', motion_ta.value)
            extra_ta.value = data.get('EXTRA_PROMPT', extra_ta.value)
            negative_ta.value = data.get('NEGATIVE_PROMPT', negative_ta.value)
            with out:
                clear_output()
                print('✓ Prompts loaded into GUI (not yet applied)')
        else:
            with out:
                clear_output()
                print('⚠️ prompts.json не найден или недоступен')

    def on_update_clicked(b):
        # Собираем итоговый PROMPT
        parts = [p.strip() for p in (base_ta.value, motion_ta.value, extra_ta.value) if p and p.strip()]
        final = ', '.join(parts)
        globals()['PROMPT'] = final
        globals()['NEGATIVE_PROMPT'] = negative_ta.value
        with out:
            clear_output()
            print('✓ PROMPT обновлён')
            print('\nPROMPT:')
            print(final)
            print('\nNEGATIVE_PROMPT:')
            print(negative_ta.value)

    preset_dropdown.observe(on_preset_change)
    save_btn.on_click(on_save_clicked)
    load_btn.on_click(on_load_clicked)
    update_btn.on_click(on_update_clicked)

    # Layout
    controls = widgets.VBox([
        preset_dropdown,
        base_ta,
        motion_ta,
        extra_ta,
        negative_ta,
        widgets.HBox([update_btn, save_btn, load_btn]),
        out
    ])

    display(controls)
    # Попробуем загрузить prompts.json в GUI при инициализации
    data = load_prompts_file()
    if data:
        base_ta.value = data.get('BASE_PROMPT', base_ta.value)
        motion_ta.value = data.get('MOTION_PROMPT', motion_ta.value)
        extra_ta.value = data.get('EXTRA_PROMPT', extra_ta.value)
        negative_ta.value = data.get('NEGATIVE_PROMPT', negative_ta.value)
        with out:
            print('✓ prompts.json загружен в GUI')

# Вызываем создание GUI
create_prompts_gui()


In [ ]:
# === ИНИЦИАЛИЗАЦИЯ ===
import os
import time
from IPython.display import FileLink, display

WORKSPACE = "/kaggle/working"
FRAMES_DIR = f"{WORKSPACE}/frames"
DATASET_DIR = "/kaggle/input/comfyui-models-gojo"

os.makedirs(FRAMES_DIR, exist_ok=True)
print("✓ Папки созданы")

In [ ]:
# === УСТАНОВКА ЗАВИСИМОСТЕЙ ===
print("🔧 Проверка и установка зависимостей...\n")

import sys
import importlib.util

def check_package(package_name):
    """Быстрая проверка наличия пакета без полного импорта"""
    return importlib.util.find_spec(package_name) is not None

# Список необходимых пакетов
packages_to_install = []

if not check_package("diffusers"):
    packages_to_install.append("diffusers[torch]")
    packages_to_install.append("transformers")
    packages_to_install.append("accelerate")
else:
    print("✓ diffusers уже установлен")

if not check_package("cv2"):
    packages_to_install.append("opencv-python")
else:
    print("✓ opencv-python уже установлен")

if USE_ANIMATEDIFF and not check_package("imageio"):
    packages_to_install.append("imageio")
    packages_to_install.append("imageio-ffmpeg")
elif USE_ANIMATEDIFF:
    print("✓ imageio уже установлен")

# Устанавливаем ipywidgets для GUI, если его нет
if not check_package("ipywidgets"):
    packages_to_install.append("ipywidgets")
else:
    print("✓ ipywidgets уже установлен")

# Устанавливаем все пакеты одной командой (через subprocess для переносимости)
if packages_to_install:
    print(f"\nУстановка: {', '.join(packages_to_install)}...")
    import subprocess
    # Используем Python interpreter для гарантированной установки в текущее окружение
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q'] + packages_to_install)
    print("✓ Установка завершена!")
    # При автоматической установке ipywidgets потребуется перезапуск kernel для корректной работы GUI
    if 'ipywidgets' in packages_to_install:
        print('\n⚠️ ipywidgets установлен — перезапустите kernel (Restart Kernel) в интерфейсе Kaggle, чтобы GUI заработал корректно.')

print("\n✓ Зависимости готовы!")

In [ ]:
#=== ПОДАВЛЕНИЕ ШУМА ОТ XLA / TensorFlow / absl ===
# Устанавливаем переменные окружения ДО импортов, чтобы избежать дублирующей регистрации CUDA-плагинов
import os
# Скрыть INFO/WARNING TensorFlow-логи (если TF подгружается косвенно)
os.environ.setdefault('TF_CPP_MIN_LOG_LEVEL', '3')
# Отключить предалокацию памяти у XLA клиента (если установлен jax/xla)
os.environ.setdefault('XLA_PYTHON_CLIENT_PREALLOCATE', 'false')
os.environ.setdefault('XLA_PYTHON_CLIENT_MEM_FRACTION', '0.0')

# Попытаемся подавить предупреждения absl уже на этапе импорта (без падений, если absl нет)
try:
    import absl.logging
    # Не писать предварительные сообщения в stderr
    absl.logging._warn_preinit_stderr = False
    absl.logging.set_verbosity(absl.logging.WARNING)
except Exception:
    pass

# === ИМПОРТ БИБЛИОТЕК ===
# Защитная проверка: если функция print_separator не была ранее определена (ячейка не выполнялась),
# определим простую совместимую версию, чтобы избежать NameError при запуске отдельной ячейки.
if "print_separator" not in globals():
    def print_separator(nl_before=True):
        SEP = '=' * 60
        if nl_before:
            print('\n' + SEP)
        else:
            print(SEP + '\n')

print_separator()
print("🎨 Загрузка библиотек (это может занять ~30 сек)...\n")

import torch
from PIL import Image

print("✓ PyTorch и PIL загружены")
print(f"✓ CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# === ЗАГРУЗКА ANIMATEDIFF PIPELINE (отдельная ячейка) ===
# Запустите эту ячейку перед ячейкой генерации, если хотите отдельно подготовить модель и сэкономить время при повторной генерации.
if 'USE_ANIMATEDIFF' in globals() and USE_ANIMATEDIFF:
    if 'pipe' in globals() and 'adapter' in globals():
        print('✓ AnimateDiff pipeline уже загружен (pipe, adapter в globals).')
    else:
        print('Загрузка AnimateDiff pipeline...')
        try:
            from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
            from diffusers.utils import export_to_video
            import torch

            adapter = MotionAdapter.from_pretrained(
                "guoyww/animatediff-motion-adapter-v1-5-2",
                torch_dtype=torch.float16
            )

            # AnimateDiff работает с SD 1.5
            pipe = AnimateDiffPipeline.from_pretrained(
                "runwayml/stable-diffusion-v1-5",
                motion_adapter=adapter,
                torch_dtype=torch.float16
            ).to("cuda")

            pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
            pipe.enable_vae_slicing()
            pipe.enable_model_cpu_offload()

            print('✓ AnimateDiff готов! (adapter & pipe загружены)')
        except Exception as _e:
            print('⚠️ Не удалось загрузить AnimateDiff pipeline:', _e)
            print('  Проверьте установку diffusers, доступ к интернету и наличие CUDA/GPU.')

In [ ]:
# === ГЕНЕРАЦИЯ КАДРОВ / АНИМАЦИИ ===
# Защитная проверка: если пользователь выполнил только эту ячейку, то установим безопасные значения по умолчанию
import os
import time
_defaults = {
    'PROMPT': "cinematic portrait of gojo satoru, white spiky hair, black blindfold, confident expression, anime style, highly detailed, 8k, professional lighting",
    'NEGATIVE_PROMPT': "blurry, deformed, low quality, watermark, text, bad anatomy, multiple heads, duplicate",
    'USE_ANIMATEDIFF': True,
    'USE_RIFE': True,
    'WIDTH': 512,
    'HEIGHT': 768,
    'NUM_FRAMES': 16,
    'STEPS': 25,
    'CFG_SCALE': 7.5,
    'FPS': 8,
    'RIFE_EXP': 4,
    'WORKSPACE': os.getcwd()
}
for _k, _v in _defaults.items():
    if _k not in globals():
        globals()[_k] = _v
        print(f"⚠️ Переменная {_k} не найдена — установлено значение по умолчанию: {_v}")

# Убедимся, что директория для кадров существует
if 'FRAMES_DIR' not in globals():
    FRAMES_DIR = f"{WORKSPACE}/frames"
os.makedirs(FRAMES_DIR, exist_ok=True)

print_separator()
print(f"🎨 {'ГЕНЕРАЦИЯ АНИМАЦИИ' if USE_ANIMATEDIFF else 'ГЕНЕРАЦИЯ КАДРОВ'}")
print_separator(nl_before=False)
print(f"Промпт: {PROMPT[:80]}...")
print(f"Размер: {WIDTH}x{HEIGHT}, Steps: {STEPS}, CFG: {CFG_SCALE}")
print(f"Кадров: {NUM_FRAMES}\n")

if USE_ANIMATEDIFF:
    # === РЕЖИМ ANIMATEDIFF - НАСТОЯЩАЯ АНИМАЦИЯ ===
    # Теперь генерация использует уже загруженный `pipe` (если он есть). Если нет — подсказываем запустить ячейку загрузки.
    if 'pipe' not in globals():
        print("⚠️ AnimateDiff pipeline не загружен. Запустите отдельную ячейку 'ЗАГРУЗКА ANИМАЦИИ PIPELINE' перед генерацией или выполните эту ячейку, чтобы загрузить его автоматически.")
        # Попробуем всё же подгрузить inline (fallback):
        try:
            from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
            from diffusers.utils import export_to_video
            import torch

            print('Загрузка AnimateDiff pipeline (fallback inline)...')
            adapter = MotionAdapter.from_pretrained(
                "guoyww/animatediff-motion-adapter-v1-5-2",
                torch_dtype=torch.float16
            )
            pipe = AnimateDiffPipeline.from_pretrained(
                "runwayml/stable-diffusion-v1-5",
                motion_adapter=adapter,
                torch_dtype=torch.float16
            ).to("cuda")
            pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
            pipe.enable_vae_slicing()
            pipe.enable_model_cpu_offload()

        except Exception as e:
            raise RuntimeError('Не удалось подгрузить AnimateDiff pipeline автоматически: ' + str(e))

    print('Генерация анимации (это займет ~3-7 минут)...')

    start_time = time.time()

    # Генерируем анимацию
    output = pipe(
        prompt=PROMPT,
        negative_prompt=NEGATIVE_PROMPT,
        num_frames=NUM_FRAMES,
        width=WIDTH,
        height=HEIGHT,
        num_inference_steps=STEPS,
        guidance_scale=CFG_SCALE,
        generator=torch.Generator("cuda").manual_seed(42)
    )

    frames = output.frames[0]
    total_gen_time = time.time() - start_time

    print(f"\n✅ Анимация готова за {total_gen_time:.1f}s!\n")

    # Сохраняем кадры
    for i, frame in enumerate(frames):
        frame.save(f"{FRAMES_DIR}/{i}.png")

    # Создаем базовое видео
    base_video = f"{WORKSPACE}/ANIMATED_BASE.mp4"
    export_to_video(frames, base_video, fps=FPS)
    print(f"✓ Базовое видео сохранено ({len(frames)} кадров, {FPS} fps)")

    del pipe, adapter
    torch.cuda.empty_cache()

else:
    # === РЕЖИМ СТАТИЧНЫХ КАДРОВ ===
    from diffusers import StableDiffusionXLPipeline

    # Загружаем модель из датасета или HuggingFace
    model_path = f"{DATASET_DIR}/sd_xl_base_1.0.safetensors"

    if not os.path.exists(model_path):
        print("⚠️ Модель не найдена в датасете, используем HuggingFace...")
        model_path = "stabilityai/stable-diffusion-xl-base-1.0"

    print("Загрузка модели SDXL...")
    pipe = StableDiffusionXLPipeline.from_single_file(
        model_path,
        torch_dtype=torch.float16,
        use_safetensors=True
    ).to("cuda")

    pipe.enable_attention_slicing()
    pipe.enable_vae_slicing()

    print("✓ Модель загружена!\n")

    # Генерируем кадры
    start_time = time.time()
    for i in range(NUM_FRAMES):
        print(f"Кадр {i+1}/{NUM_FRAMES}...", end=" ")

        generator = torch.Generator(device="cuda").manual_seed(42 + i)

        image = pipe(
            prompt=PROMPT,
            negative_prompt=NEGATIVE_PROMPT,
            width=WIDTH,
            height=HEIGHT,
            num_inference_steps=STEPS,
            guidance_scale=CFG_SCALE,
            generator=generator
        ).images[0]

        image.save(f"{FRAMES_DIR}/{i}.png")
        print(f"✓ ({time.time() - start_time:.1f}s)")

    total_gen_time = time.time() - start_time
    print(f"\n✅ {NUM_FRAMES} кадров за {total_gen_time:.1f}s!\n")

    del pipe
    torch.cuda.empty_cache()

print(f"✓ Кадры сохранены в: {FRAMES_DIR}")

In [ ]:
# === UPSCALE С REAL-ESRGAN (ОПЦИОНАЛЬНО) ===
print_separator()
print("📈 АПСКЕЙЛ КАДРОВ С REAL-ESRGAN")
print_separator(nl_before=False)

# Защитные значения, чтобы ячейка была idempotent при запуске отдельно
import os
if 'WORKSPACE' not in globals():
    WORKSPACE = os.getcwd()
    print(f"⚠️ Переменная WORKSPACE не найдена — использую: {WORKSPACE}")
if 'FRAMES_DIR' not in globals():
    FRAMES_DIR = f"{WORKSPACE}/frames"
    print(f"⚠️ Переменная FRAMES_DIR не найдена — использую: {FRAMES_DIR}")
if 'DATASET_DIR' not in globals():
    DATASET_DIR = f"{WORKSPACE}/dataset"
    print(f"⚠️ Переменная DATASET_DIR не найдена — использую: {DATASET_DIR}")

os.makedirs(FRAMES_DIR, exist_ok=True)
os.makedirs(DATASET_DIR, exist_ok=True)

upscale_model = f"{DATASET_DIR}/{UPSCALE_MODEL_NAME}"

if os.path.exists(upscale_model):
    # Клонируем Real-ESRGAN если еще не установлен
    if not os.path.exists(f"{WORKSPACE}/Real-ESRGAN"):
        print("Установка Real-ESRGAN...")
        import subprocess, os
        subprocess.check_call(["git", "clone", "https://github.com/xinntao/Real-ESRGAN", f"{WORKSPACE}/Real-ESRGAN"])
    # Переходим в папку репозитория
    os.chdir(f"{WORKSPACE}/Real-ESRGAN")
    # Устанавливаем зависимости через pip (subprocess)
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'basicsr', 'facexlib', 'gfpgan', 'realesrgan'])

    # Копируем модель в weights
    import shutil
    os.makedirs('weights', exist_ok=True)
    shutil.copy(upscale_model, 'weights/')

    # Апскейл каждого кадра (запускаем скрипт Real-ESRGAN через Python)
    print("\nЗапуск апскейла...")
    subprocess.check_call([sys.executable, 'inference_realesrgan.py', '-n', '4x-UltraSharp', '-i', FRAMES_DIR, '-o', FRAMES_DIR + '_upscaled', '--fp32'])

    # Заменяем оригинальные кадры апскейленными
    shutil.rmtree(FRAMES_DIR, ignore_errors=True)
    shutil.move(FRAMES_DIR + '_upscaled', FRAMES_DIR)

    # Переименовываем обратно (Real-ESRGAN добавляет суффикс)
    import glob
    upscaled_files = sorted(glob.glob(f"{FRAMES_DIR}/*_out.png"))
    for i, filepath in enumerate(upscaled_files):
        os.rename(filepath, f"{FRAMES_DIR}/{i}.png")

    print("\n✓ Апскейл завершен!")
    os.chdir(WORKSPACE)
else:
    # Попытаемся найти модель в нескольких стандартных местах и дать пользователю понятную инструкцию
    candidates = [
        os.path.join(DATASET_DIR, UPSCALE_MODEL_NAME),
        os.path.join(WORKSPACE, 'dataset', UPSCALE_MODEL_NAME),
        os.path.join(WORKSPACE, 'models', UPSCALE_MODEL_NAME),
        '/kaggle/input/comfyui-models-gojo/' + UPSCALE_MODEL_NAME,
        os.path.expanduser('~/models/' + UPSCALE_MODEL_NAME),
    ]
    found = None
    for p in candidates:
        if p and os.path.exists(p):
            found = p
            break

    if found:
        print(f"ℹ️ Модель Upscale найдена: {found} — обновляю путь и запускаю апскейл.")
        upscale_model = found
        # Повторяем логику запуска апскейла (минимально): клонирование/копирование/запуск
        if not os.path.exists(f"{WORKSPACE}/Real-ESRGAN"):
            print("Установка Real-ESRGAN...")
            import subprocess, os
            subprocess.check_call(["git", "clone", "https://github.com/xinntao/Real-ESRGAN", f"{WORKSPACE}/Real-ESRGAN"])
        os.chdir(f"{WORKSPACE}/Real-ESRGAN")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'basicsr', 'facexlib', 'gfpgan', 'realesrgan'])
        import shutil, glob
        os.makedirs('weights', exist_ok=True)
        shutil.copy(upscale_model, 'weights/')
        print("\nЗапуск апскейла...")
        subprocess.check_call([sys.executable, 'inference_realesrgan.py', '-n', '4x-UltraSharp', '-i', FRAMES_DIR, '-o', FRAMES_DIR + '_upscaled', '--fp32'])
        shutil.rmtree(FRAMES_DIR, ignore_errors=True)
        shutil.move(FRAMES_DIR + '_upscaled', FRAMES_DIR)
        upscaled_files = sorted(glob.glob(f"{FRAMES_DIR}/*_out.png"))
        for i, filepath in enumerate(upscaled_files):
            os.rename(filepath, f"{FRAMES_DIR}/{i}.png")
        print("\n✓ Апскейл завершен!")
        os.chdir(WORKSPACE)
    else:
        print("⚠️ Upscale модель не найдена — пропускаем апскейл.")
        print("   Проверьте, что файл '4x-UltraSharp.pth' присутствует в одной из директорий:")
        for c in candidates:
            print(f"     - {c}")
        print("   Или поместите модель в папку вашего датасета и установите DATASET_DIR в начале ноутбука, например:")
        print("     DATASET_DIR = '/kaggle/input/comfyui-models-gojo'")
        print("   После размещения модели перезапустите kernel и выполните ячейки заново.")

In [ ]:
# === RIFE ИНТЕРПОЛЯЦИЯ ===
print_separator()
print("🎞️ ИНТЕРПОЛЯЦИЯ КАДРОВ")
print_separator(nl_before=False)

if not USE_RIFE:
    print(f"⚠️ RIFE отключен, создаем видео из {NUM_FRAMES} кадров...\n")
    import subprocess
    subprocess.check_call(["ffmpeg", "-framerate", str(FPS), "-i", f"{FRAMES_DIR}/%d.png", "-c:v", "libx264", "-pix_fmt", "yuv420p", "-preset", "fast", f"{WORKSPACE}/GOJO_OUTPUT.mp4", "-y", "-loglevel", "error"])

    final_video = f"{WORKSPACE}/GOJO_OUTPUT.mp4"
    if os.path.exists(final_video):
        file_size = os.path.getsize(final_video) / 1024 / 1024
        print(f"\n✅ ГОТОВО! ({file_size:.1f} MB)")
        display(FileLink(final_video))
else:
    print(f"Интерполяция {NUM_FRAMES} → {NUM_FRAMES * (2**RIFE_EXP)} кадров с RIFE...\n")

    # Устанавливаем Practical-RIFE
    if not os.path.exists(f"{WORKSPACE}/RIFE"):
        print("Установка RIFE...")
        import subprocess, shutil, os, zipfile, glob
        shutil.rmtree(f"{WORKSPACE}/RIFE", ignore_errors=True)
        subprocess.check_call(["git", "clone", "https://github.com/hzwer/Practical-RIFE", f"{WORKSPACE}/RIFE"])

    os.chdir(f"{WORKSPACE}/RIFE")

    # Устанавливаем зависимости RIFE
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'scikit-video'])

    # Скачиваем модель RIFE v4.26
    if not os.path.exists("train_log/flownet.pkl"):
        print("Скачивание модели RIFE v4.26...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'gdown'])
        os.makedirs('train_log', exist_ok=True)
        # Имя архива с моделью
        TRAIN_LOG_ZIP = "train_log.zip"
        # Скачиваем архив через gdown
        subprocess.check_call(["gdown", "1gViYvvQrtETBgU1w8axZSsr7YUuw31uy", "-O", TRAIN_LOG_ZIP])

        model_downloaded = False

        if os.path.exists(TRAIN_LOG_ZIP) and os.path.getsize(TRAIN_LOG_ZIP) > 1000000:
            # Распаковываем zip через zipfile
            with zipfile.ZipFile(TRAIN_LOG_ZIP, 'r') as zf:
                zf.extractall('.')

            # Ищем модель
            if os.path.exists("train_log/flownet.pkl"):
                model_downloaded = True
                print("✓ Модель RIFE готова!")
            else:
                # Ищем в других местах с помощью glob
                result = glob.glob('**/flownet.pkl', recursive=True)
                if result:
                    shutil.copy(result[0], 'train_log/flownet.pkl')
                    model_downloaded = True
                    print("✓ Модель RIFE готова!")

        if not model_downloaded:
            print("⚠️ Не удалось скачать модель RIFE")
            print("Создаем простое видео из кадров...")
            subprocess.check_call(["ffmpeg", "-framerate", str(FPS), "-i", f"{FRAMES_DIR}/%d.png", "-c:v", "libx264", "-pix_fmt", "yuv420p", "-preset", "fast", f"{WORKSPACE}/GOJO_OUTPUT.mp4", "-y", "-loglevel", "error"])
    else:
        print("✓ Модель RIFE уже загружена")
        model_downloaded = True

    if model_downloaded:
        # Запускаем RIFE интерполяцию
        print(f"\nЗапуск интерполяции (exp={RIFE_EXP})...")

        # Запускаем inference script через Python
        subprocess.check_call([sys.executable, 'inference_video.py', '--img', FRAMES_DIR, '--exp', str(RIFE_EXP), '--output', f"{WORKSPACE}/GOJO_OUTPUT.mp4", '--UHD'])

        # Проверяем результат
        final_video = f"{WORKSPACE}/GOJO_OUTPUT.mp4"
        if os.path.exists(final_video):
            file_size = os.path.getsize(final_video) / 1024 / 1024
            print(f"\n🎉 ГОТОВО! Видео {NUM_FRAMES * (2**RIFE_EXP)} кадров ({file_size:.1f} MB)")
            display(FileLink(final_video))
        else:
            print("\n⚠️ RIFE не создал видео, создаем fallback...")
            subprocess.check_call(["ffmpeg", "-framerate", str(FPS), "-i", f"{FRAMES_DIR}/%d.png", "-c:v", "libx264", "-pix_fmt", "yuv420p", "-preset", "fast", f"{WORKSPACE}/GOJO_OUTPUT.mp4", "-y", "-loglevel", "error"])

            if os.path.exists(final_video):
                file_size = os.path.getsize(final_video) / 1024 / 1024
                print(f"\n✅ ГОТОВО! ({file_size:.1f} MB)")
                display(FileLink(final_video))
    
    os.chdir(WORKSPACE)

In [ ]:
# === ИТОГИ ===
# Защитная проверка: если переменная total_gen_time не определена (ячейка генерации не запускалась),
# ставим безопасное значение 0.0
if 'total_gen_time' not in globals():
    total_gen_time = 0.0

print_separator()
print("📋 ИТОГИ ГЕНЕРАЦИИ")
print_separator(nl_before=False)
print(f"Режим: {'AnimateDiff (анимация)' if USE_ANIMATEDIFF else 'Статичные кадры'}")
print(f"Промпт: {PROMPT[:50]}...")
print(f"Размер: {WIDTH}x{HEIGHT}")
print(f"Кадров: {NUM_FRAMES}" + (f" → {NUM_FRAMES * (2**RIFE_EXP)}" if USE_RIFE else ""))
print(f"Время генерации: {total_gen_time:.1f}s")

if USE_ANIMATEDIFF:
    print(f"\n{'='*60}")
    print("💡 СОВЕТЫ ДЛЯ УЛУЧШЕНИЯ АНИМАЦИИ")
    print(f"{'='*60}")
    print("Добавьте в промпт:")
    print("  • 'turning head', 'blinking', 'hair flowing'")
    print("  • 'smooth motion', 'cinematic camera movement'")
    print("  • 'dynamic pose', 'wind blowing'")
    print("\nДобавьте в negative:")
    print("  • 'static', 'frozen', 'choppy animation'")
    print("  • 'stiff', 'rigid', 'still image'")

print(f"\n{'='*60}")
print("✅ ВСЁ ГОТОВО!")
print(f"{'='*60}")


In [ ]:
# === UNLOAD PIPE (освобождение pipeline и VRAM) ===
# Нажмите кнопку, чтобы удалить `pipe` и `adapter` из globals и очистить VRAM.
def unload_pipe():
    """Удаляет pipe/adapter из глобальной области и очищает VRAM (если доступен torch)."""
    import gc
    removed = []
    try:
        if 'pipe' in globals():
            try:
                del globals()['pipe']
                removed.append('pipe')
            except Exception:
                pass
        if 'adapter' in globals():
            try:
                del globals()['adapter']
                removed.append('adapter')
            except Exception:
                pass
        # Попробуем освободить GPU память
        try:
            import torch
            torch.cuda.empty_cache()
            removed.append('cuda_cache_cleared')
        except Exception:
            pass
        # Общая уборка памяти
        gc.collect()
        print(f"✓ Удалено: {', '.join(removed) if removed else 'ничего не найдено'}")
    except Exception as e:
        print('⚠️ Ошибка при выгрузке pipe:', e)

# Показываем кнопку если ipywidgets доступен, иначе показываем инструкцию
try:
    import ipywidgets as widgets
    from IPython.display import display
    btn_unload = widgets.Button(description='Unload pipe (free VRAM)', button_style='warning')
    def _on_unload_click(b):
        unload_pipe()
    btn_unload.on_click(_on_unload_click)
    display(btn_unload)
except Exception:
    print('\nℹ️ Для быстрого освобождения памяти выполните в ячейке: unload_pipe()')

